In [1]:
import numpy as np
import xarray as xr
import rasterio
from rasterio.transform import from_origin

In [4]:
def netcdf_variable_to_geotiff(
    input_path, var_name, output_path, lat_name="lat", lon_name="lon", crs="EPSG:4326"
):
    """
    Reads a 2D variable from a NetCDF file and writes it to a GeoTIFF.

    Parameters:
    - input_path: Path to the input NetCDF file.
    - var_name: Name of the 2D variable in the NetCDF to export.
    - output_path: Path where the output GeoTIFF will be saved.
    - lat_name: Name of the latitude coordinate (1D array).
    - lon_name: Name of the longitude coordinate (1D array).
    - crs: Coordinate reference system for the output GeoTIFF.
    """
    # Open the NetCDF dataset
    ds = xr.open_dataset(input_path)

    # Extract the 2D data array by variable name
    try:
        data_arr = ds[var_name]
    except KeyError:
        raise KeyError(f"Variable '{var_name}' not found in dataset.")

    # Ensure it's 2D
    if data_arr.ndim != 2:
        raise ValueError(f"Variable '{var_name}' must be 2D; got {data_arr.ndim}D.")

    # Get coordinate arrays
    lats = ds[lat_name].values
    lons = ds[lon_name].values
    if lats.ndim != 1 or lons.ndim != 1:
        raise ValueError("Latitude and longitude must be 1D arrays.")

    data = data_arr.values

    # Compute resolution
    lat_res = abs(lats[1] - lats[0])
    lon_res = abs(lons[1] - lons[0])

    # Define affine transform (assumes regular grid)
    transform = from_origin(
        west=lons.min() - lon_res / 2,
        north=lats.max() + lat_res / 2,
        xsize=lon_res,
        ysize=lat_res,
    )

    # Write GeoTIFF
    with rasterio.open(
        output_path,
        "w",
        driver="GTiff",
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(data, 1)

    print(f"GeoTIFF saved to: {output_path}")

In [7]:
### run
input_nc = r"C:\Users\TyHow\Downloads\fixeddh2.nc"
output_tif = r"C:\Users\TyHow\Downloads\stripping_ratio_fixed2.tif"
layer_idx  = 0  


netcdf_variable_to_geotiff(
    input_path=input_nc,
    output_path=output_tif,
    lat_name="y",
    lon_name="x",
    var_name="dhfixed2_stripping_ratio"
)

GeoTIFF saved to: C:\Users\TyHow\Downloads\stripping_ratio_fixed2.tif
